**Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c)2017. P. Fernandez, C. Cooper. Based on [BEM++](https://bempp.com/), also under MIT license.**

# Solvation energy of a protein with a continuum electrostatic model

## Background

The energy required to dissolve a molecule (that is, moving it from vacuum into a solvent), is known as the solvation energy. This quantity is of interest in several biomolecular applications, like binding, stability, pKa calculations, among other. In a continuum electrostatic description, the dissolved biomolecule can be modeled with two dielectric regions interfaced by a molecular surface: an outer *solvent* region, usually comprised of water and salt ions, and an inner *protein* region, where the solvent cannot access, which contains point charges at the locations of the atoms.  

<img src="sketch_protein.png", style="width:300px">

Mathematically, the model describes the electrostatic potential with a system of partial differential equations. The protein region ($\Omega_1$) is governed by a Poisson equation with a source term due to the partial charges of the atoms, whereas the solvent region ($\Omega_2$) uses a Poisson-Boltzmann equation (also known as modified Helmholtz equation) to account for the presence of salt. Then, we can calculate the electrostatic potential $\phi$ by

**Interior**
$$ \nabla^2 \phi_1 (x) = \sum_k q_k \delta (x_k)  $$
**Exterior**
$$ \nabla^2 \phi_2 (x) - \kappa \phi_2 (x) = 0  $$

with $q_k$ and $x_k$ as charge and position of each molecule atom, $\kappa$ is the inverse of the Debye length. On the molecular surface, we impose the following interface conditions:

$$ \phi_{1} = \phi_{2} 
\hspace{40pt}
\epsilon_{1} \frac{\partial \phi_{1}}{\partial \textrm{ n}} = \epsilon_{2} \frac{\partial \phi_{2}}{\partial \textrm{ n}} $$

Following the work by Yoon and Lenhoff [1], the system of PDEs with interface conditions can be formulated in terms of the following boundary integral equations:

**Interior**
$$ \frac{\phi_1(x)}{2} + 
\int_\Gamma \frac{\partial G_L}{\partial \textrm{ n}}(x,x_\Gamma) \phi_1 (x_\Gamma) \textrm{d} x_\Gamma - 
\int_\Gamma G_L\left( x,x_\Gamma \right) \frac{\partial \phi_1}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma
= \sum_k \frac{q_k}{4\pi |x-x_k|} $$

**Exterior**
$$ \frac{\phi_1(x)}{2} - 
\int_\Gamma \frac{\partial G_Y}{\partial \textrm{ n}}(x,x_\Gamma) \phi_1 (x_\Gamma) \textrm{d} x_\Gamma + 
\frac{\epsilon_1}{\epsilon_2}\int_\Gamma G_Y(x,x_\Gamma) \frac{\partial \phi_1}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma 
= 0$$

In terms of boundary operators, this can be written as

$$
\begin{bmatrix}
    \frac{I}{2} + K_L &  -V_L \\
    \frac{I}{2} - K_Y &  \frac{\epsilon_1}{\epsilon_2}V_Y
\end{bmatrix}
\left\{
\begin{matrix}
    \phi_1 \\
    \frac{\partial \phi_1}{\partial \textrm{ n}}
\end{matrix}
\right\}
= \left\{
\begin{matrix}
    \sum_k q_k \\
    0
\end{matrix}
\right\}
$$

where $V_L$ and $K_L$ are the single and double layer operators for the Laplace kernel (Poisson equation) and $V_Y$ and $K_Y$ are the corresponding operators for the Yukawa kernel (Poisson-Boltzmann or modified Helmholtz equation).

From there, we can compute the *reaction potential* ($\phi_\text{reac} = \phi_1-\phi_\text{Coulomb}$) at the locations of the atoms with

$$\phi_\text{reac} = 
\int_\Gamma G_L(x_k,x_\Gamma) \frac{\partial \phi}{\partial \textrm{ n}} (x_\Gamma) \textrm{d} x_\Gamma - 
\int_\Gamma \frac{\partial G_L}{\partial \textrm{ n}}(x_k,x_\Gamma) \phi (x_\Gamma) \textrm{d} x_\Gamma $$

which can be written in terms of operators as

$$\phi_k = K_L\left[ \frac{\partial \phi}{\partial n} \right] - V_L \left[ \phi \right] $$,
       
to then obtain the solvation energy as
       
$$ \Delta G_\text{solv} = \frac{1}{2} \sum_{k=1}^{N_q} q_k \phi_\text{reac}(x_k) $$

## Example

### Molecular structure and mesh

As an example, we will calculate the solvation energy of the bovine pancreatic trypsin inhibitor. The molecular crystal structure is available the protein data bank ([PDB code 5PTI](https://www.rcsb.org/pdb/explore.do?structureId=5PTI)), and we obtained the atomic radii and charges with [pdb2pqr](http://www.poissonboltzmann.org/) [2], resulting in a `.pqr` file.

There are several molecular surface definitions to put the interface conditions. In this case, we use the *solvent-excluded surface* (SES), which is the result of rolling a spherical probe of the size of a water molecule around the protein, and tracking the contact points. We mesh the SES with [`msms`](http://mgl.scripps.edu/people/sanner/html/msms_home.html) [3], and reformat the result to a `.msh` file with `GridFactory()`. 

First, let's import the required libraries and mesh

In [1]:
import bempp.api
#bempp.api.set_ipython_notebook_viewer()

grid = bempp.api.import_grid('5pti_d1.msh')
#grid.plot()

<img src="5pti_mesh.png", style="width:400px">

and read the `.pqr` file for the charges.

In [2]:
import numpy as np
q, x_q = np.array([]), np.empty((0,3))

ep_in = 4.
ep_ex = 80.
k = 0.125

# Read charges and coordinates from the .pqr file
molecule_file = open('5pti.pqr', 'r').read().split('\n')
for line in molecule_file:
    line = line.split()
    if len(line)==0 or line[0]!='ATOM': continue
    q = np.append( q, float(line[8]))
    x_q = np.vstack(( x_q, np.array(line[5:8]).astype(float) ))

### Right-hand side

With that, we can compute the potential due to the charges on the boundary, required for the right-hand side

In [3]:
# Function to calculate the potential by charges at boundary
def charges_fun(x, n, domain_index, result):
    global q, x_q, ep_in
    result[:] = np.sum(q/np.linalg.norm( x - x_q, axis=1 ))/(4*np.pi*ep_in)

dirichl_space = bempp.api.function_space(grid, "DP", 0)
neumann_space = bempp.api.function_space(grid, "DP", 0)

charged_grid_fun = bempp.api.GridFunction(dirichl_space, fun=charges_fun)
#charged_grid_fun.plot()

rhs = np.concatenate([charged_grid_fun.coefficients, 
                      np.zeros(neumann_space.global_dof_count)])


### Operators and Matrix

Next, we generate the $2\times 2$ block matrix with the single and double layer operators of the Laplace and Yukawa kernels 

In [4]:
# Define Operators
from bempp.api.operators.boundary import sparse, laplace, modified_helmholtz
identity = sparse.identity(dirichl_space, dirichl_space, dirichl_space)
slp_in   = laplace.single_layer(neumann_space, dirichl_space, dirichl_space)
dlp_in   = laplace.double_layer(dirichl_space, dirichl_space, dirichl_space)
slp_out  = modified_helmholtz.single_layer(neumann_space, dirichl_space, dirichl_space, k)
dlp_out  = modified_helmholtz.double_layer(dirichl_space, dirichl_space, dirichl_space, k)

# Matrix Assembly
blocked = bempp.api.BlockedOperator(2, 2)
blocked[0, 0] = 0.5*identity + dlp_in
blocked[0, 1] = -slp_in
blocked[1, 0] = 0.5*identity - dlp_out
blocked[1, 1] = (ep_in/ep_ex)*slp_out
op_discrete = blocked.strong_form()

### Solver

We now use `gmres` from `scipy` to solve the system. 

In [5]:
import inspect
from scipy.sparse.linalg import gmres

array_it, array_frame, it_count = np.array([]), np.array([]), 0
def iteration_counter(x):
        global array_it, array_frame, it_count
        it_count += 1
        frame = inspect.currentframe().f_back
        array_it = np.append(array_it, it_count)
        array_frame = np.append(array_frame, frame.f_locals["resid"])
        #print "It: {0} Error {1:.2E}".format(it_count, frame.f_locals["resid"])        

x, info = gmres(op_discrete, rhs, callback=iteration_counter, tol=1e-3, maxiter=500, restart = 1000)

print("The linear system was solved in {0} iterations".format(it_count))

The linear system was solved in 107 iterations


The two following `GridFunction` calls store the calculated boundary potential data (separated by $\phi$ and $\frac{\partial \phi}{\partial n}$) for visualization purposes. 

In [6]:
solution_dirichl = bempp.api.GridFunction(dirichl_space, 
                                          coefficients=x[:dirichl_space.global_dof_count])
solution_neumann = bempp.api.GridFunction(neumann_space, 
                                          coefficients=x[dirichl_space.global_dof_count:])
#solution_dirichl.plot()

which should give something like:

<img src="5pti_solu.png", style="width:300px">

### Energy calculaton

To compute $\phi_\text{reac}$ at the atoms locations, we use `operators.potential` to then multiply by the charge and add to compute $\Delta G_\text{solv}$. The `332.064` term is just a unit conversion constant.

In [7]:
slp_q = bempp.api.operators.potential.laplace.single_layer(neumann_space, x_q.transpose())
dlp_q = bempp.api.operators.potential.laplace.double_layer(dirichl_space, x_q.transpose())
phi_q = slp_q*solution_neumann - dlp_q*solution_dirichl

# total dissolution energy applying constant to get units [kcal/mol]
total_energy = 2*np.pi*332.064*np.sum(q*phi_q).real
print("Total solvation energy: {:7.2f} [kcal/Mol]".format(total_energy))

Total solvation energy: -352.67 [kcal/Mol]


## References
[1] Yoon, B. J., & Lenhoff, A. M. (1990). A boundary element method for molecular electrostatics with electrolyte effects. *Journal of Computational Chemistry*, 11(9), 1080-1086.

[2] Dolinsky, T. J., Nielsen, J. E., McCammon, J. A., & Baker, N. A. (2004). PDB2PQR: an automated pipeline for the setup of Poisson–Boltzmann electrostatics calculations. *Nucleic acids research*, 32(suppl_2), W665-W667.

[3] Sanner, M. F., Olson, A. J., & Spehner, J. C. (1995, September). Fast and robust computation of molecular surfaces. In *Proceedings of the eleventh annual symposium on Computational geometry* (pp. 406-407). ACM.